# Algorithm 8: Triangle Multiplication (Boltz)

Encodes triangle inequality constraints in pair representations.

## Source Code Location
- **File**: `Boltz-Ref-src/boltz-official/src/boltz/model/layers/triangular_mult.py`

In [ ]:
import numpy as np
np.random.seed(42)

def layer_norm(x, eps=1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.var(x, axis=-1, keepdims=True)
    return (x - mean) / np.sqrt(var + eps)

def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

In [ ]:
def triangle_multiplication_outgoing(z, c=64):
    """
    Triangle Multiplication (Outgoing edges).
    
    For edge (i,j), aggregates information from edges (i,k) and (j,k).
    
    Args:
        z: Pair representation [N, N, c_z]
        c: Hidden dimension
    
    Returns:
        Update [N, N, c_z]
    """
    N = z.shape[0]
    c_z = z.shape[-1]
    
    print(f"Triangle Multiplication (Outgoing)")
    print(f"="*50)
    print(f"Pair: [{N}, {N}, {c_z}]")
    
    z_norm = layer_norm(z)
    
    # Gated projections
    W_a = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_ag = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_b = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_bg = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_g = np.random.randn(c_z, c_z) * (c_z ** -0.5)
    
    a = sigmoid(z_norm @ W_ag) * (z_norm @ W_a)
    b = sigmoid(z_norm @ W_bg) * (z_norm @ W_b)
    g = sigmoid(z_norm @ W_g)
    
    # Triangle: sum_k a[i,k] * b[j,k]
    output = np.einsum('ikc,jkc->ijc', a, b)
    output = layer_norm(output)
    
    W_o = np.random.randn(c, c_z) * (c ** -0.5)
    output = (output @ W_o) * g
    
    print(f"Output: {output.shape}")
    return output

def triangle_multiplication_incoming(z, c=64):
    """
    Triangle Multiplication (Incoming edges).
    
    For edge (i,j), aggregates from edges (k,i) and (k,j).
    """
    N = z.shape[0]
    c_z = z.shape[-1]
    
    print(f"Triangle Multiplication (Incoming)")
    print(f"="*50)
    
    z_norm = layer_norm(z)
    
    W_a = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_ag = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_b = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_bg = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_g = np.random.randn(c_z, c_z) * (c_z ** -0.5)
    
    a = sigmoid(z_norm @ W_ag) * (z_norm @ W_a)
    b = sigmoid(z_norm @ W_bg) * (z_norm @ W_b)
    g = sigmoid(z_norm @ W_g)
    
    # Triangle: sum_k a[k,i] * b[k,j]
    output = np.einsum('kic,kjc->ijc', a, b)
    output = layer_norm(output)
    
    W_o = np.random.randn(c, c_z) * (c ** -0.5)
    output = (output @ W_o) * g
    
    print(f"Output: {output.shape}")
    return output

In [ ]:
# Test
print("Test: Triangle Multiplication")
print("="*60)

N = 32
c_z = 64

z = np.random.randn(N, N, c_z) * 0.1

out_out = triangle_multiplication_outgoing(z, c=32)
print(f"Outgoing finite: {np.isfinite(out_out).all()}")

out_in = triangle_multiplication_incoming(z, c=32)
print(f"Incoming finite: {np.isfinite(out_in).all()}")

## Key Insights

1. **Geometric Constraint**: Encodes triangle inequality
2. **Outgoing/Incoming**: Two complementary edge directions
3. **Gated Projections**: Controlled information flow
4. **Sum over k**: Aggregates all possible third vertices